### Data Acquisition

In [10]:
import pandas as pd
from sqlalchemy import create_engine
import requests
from bs4 import BeautifulSoup as bs

#### 1.1 Tables (.db)

In [11]:
sqlitepath = '../data/raw/raw_data_project_m1.db'
engine = create_engine(f'sqlite:///{sqlitepath}')

In [12]:
# Table summary
pd.read_sql_query("SELECT * from sqlite_master WHERE type = 'table'", engine)

,type,name,tbl_name,rootpage,sql
0,table,personal_info,personal_info,2,CREATE TABLE personal_info (\n\tuuid TEXT NOT ...
1,table,country_info,country_info,401,CREATE TABLE country_info (\n\tuuid TEXT NOT N...
2,table,career_info,career_info,762,CREATE TABLE career_info (\n\tuuid TEXT NOT NU...
3,table,poll_info,poll_info,1168,CREATE TABLE poll_info (\n\tuuid TEXT NOT NULL...


Get the tables -all in one

In [ ]:
#all query
df_data_project = pd.read_sql_query("""SELECT * FROM career_info  
inner join poll_info on career_info.uuid  
inner join country_info on country_info.uuid
inner join personal_info on personal_info.uuid""",engine)
data_raw = pd.DataFrame(df_data_project)
data_raw

Get tables - one by one

In [14]:
# Table acquisition
df_personal_info = pd.read_sql_table('personal_info', engine)
df_country_info = pd.read_sql_table('country_info', engine)
df_career_info = pd.read_sql_table('career_info', engine)
df_poll_info = pd.read_sql_table('poll_info', engine)

Tables summary

In [15]:
print(f'Table: Personal info\nRows: {df_personal_info.shape[0]} \nColumns: {df_personal_info.shape[1]}\n\nNull values:\n{df_personal_info.isnull().sum()}')

Table: Personal info
Rows: 9649 
Columns: 5

Null values:
uuid                0
age                 0
gender              0
dem_has_children    0
age_group           0
dtype: int64


In [16]:
# save in csv 
df_personal_info.to_csv('../data/raw/personal_info.csv')

In [17]:
##### 1.1.2 Country info table

In [18]:
print(f'Table: Country info\nRows: {df_country_info.shape[0]} \nColumns: {df_country_info.shape[1]}\n\nNull values:\n{df_country_info.isnull().sum()}')

Table: Country info
Rows: 9649 
Columns: 3

Null values:
uuid            0
country_code    0
rural           0
dtype: int64


In [19]:
df_country_info.to_csv('../data/raw/country_info.csv')

In [20]:
print(f'Table: Career info\nRows: {df_career_info.shape[0]} \nColumns: {df_career_info.shape[1]}\n\nNull values:\n{df_career_info.isnull().sum()}')

Table: Career info
Rows: 9649 
Columns: 4

Null values:
uuid                      0
dem_education_level     663
dem_full_time_job         0
normalized_job_code    3947
dtype: int64


Career table has Null values for dem_education_level (663) and normalized_job_code (3947) - \n 
We need to take care of the ones from normalized_job_code before we save

In [21]:
# Easiest way is Null to string 'None'
df_career_info['normalized_job_code'].fillna('None', inplace=True)

In [22]:
df_career_info.to_csv('../data/raw/career_info.csv')

In [23]:
print(f'Table: Poll info\nRows: {df_poll_info.shape[0]} \nColumns: {df_poll_info.shape[1]}\n\nNull values:\n{df_poll_info.isnull().sum()}')

Table: Poll info
Rows: 9649 
Columns: 6

Null values:
uuid                                                   0
question_bbi_2016wave4_basicincome_awareness           0
question_bbi_2016wave4_basicincome_vote                0
question_bbi_2016wave4_basicincome_effect              0
question_bbi_2016wave4_basicincome_argumentsfor        0
question_bbi_2016wave4_basicincome_argumentsagainst    0
dtype: int64


In [24]:
df_poll_info.to_csv('../data/raw/poll_info.csv')

#### 1.2 Check final outputs

In [25]:
display(df_career_info.head(5))

,uuid,dem_education_level,dem_full_time_job,normalized_job_code
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,no,no,None
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,high,yes,861a9b9151e11362eb3c77ca914172d0
2,83127080-da3d-0133-c74f-0a81e8b09a82,None,no,None
3,15626d40-db13-0133-ea5c-0a81e8b09a82,high,yes,049a3f3a2b5f85cb2971ba77ad66e10c
4,24954a70-db98-0133-4a64-0a81e8b09a82,high,yes,f4b2fb1aa40f661488e2782b6d57ad2f


In [26]:
display(df_country_info.head(5))

,uuid,country_code,rural
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,AT,countryside
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,AT,urban
2,83127080-da3d-0133-c74f-0a81e8b09a82,AT,city
3,15626d40-db13-0133-ea5c-0a81e8b09a82,AT,Country
4,24954a70-db98-0133-4a64-0a81e8b09a82,AT,city


In [27]:
display(df_personal_info.head(5))

,uuid,age,gender,dem_has_children,age_group
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,61 years old,male,NO,40_65
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,57 years old,male,yES,40_65
2,83127080-da3d-0133-c74f-0a81e8b09a82,32 years old,male,nO,26_39
3,15626d40-db13-0133-ea5c-0a81e8b09a82,45 years old,Male,YES,40_65
4,24954a70-db98-0133-4a64-0a81e8b09a82,41 years old,Fem,yES,40_65


In [28]:
display(df_poll_info.head(5))

,uuid,question_bbi_2016wave4_basicincome_awareness,question_bbi_2016wave4_basicincome_vote,question_bbi_2016wave4_basicincome_effect,question_bbi_2016wave4_basicincome_argumentsfor,question_bbi_2016wave4_basicincome_argumentsagainst
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,I know something about it,I would not vote,None of the above,None of the above,None of the above
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,I understand it fully,I would probably vote for it,A basic income would not affect my work choices,It increases appreciation for household work a...,It might encourage people to stop working
2,83127080-da3d-0133-c74f-0a81e8b09a82,I have heard just a little about it,I would not vote,‰Û_ gain additional skills,It creates more equality of opportunity,Foreigners might come to my country and take a...
3,15626d40-db13-0133-ea5c-0a81e8b09a82,I have heard just a little about it,I would probably vote for it,‰Û_ work less,It reduces anxiety about financing basic needs,None of the above
4,24954a70-db98-0133-4a64-0a81e8b09a82,I have heard just a little about it,I would probably vote for it,None of the above,It reduces anxiety about financing basic needs,It is impossible to finance | It might encoura...


#### 1.2 API. We will use the API from the Open Skills Project

First we need to get the job codes [unique values] from the career info df so we can call merge with api - normalized job codes

Option 1

In [29]:
jobs_api = []
job_codes = set(df_career_info['normalized_job_code'])
# job_codes = df_career_info['normalized_job_code'].unique().tolist()

In [30]:
# get job codes on a loop
for i in job_codes:
    response = requests.get(f'http://api.dataatwork.org/v1/jobs/{i}')
    jobs_json = response.json()
    jobs_api.append(jobs_json)
# jobs_api

In [31]:
jobs = pd.DataFrame(jobs_api).rename(columns = {'uuid':'normalized_job_code'})[['normalized_job_code', 'title', 'normalized_job_title']]

In [32]:
jobs.shape

(157, 3)

Option 2

In [33]:
jobs_api2 = []
job_codes2 = df_career_info['normalized_job_code'].unique().tolist()
for i in job_codes2:
    response = requests.get(f'http://api.dataatwork.org/v1/jobs/{i}')
    jobs_json2 = response.json()
    jobs_api2.append(jobs_json2)

In [34]:
jobs2 = pd.DataFrame(jobs_api2).rename(columns = {'uuid':'normalized_job_code'})[['normalized_job_code', 'title', 'normalized_job_title']]

In [35]:
jobs2.shape

(157, 3)

Option 1 (314, 3) vs Option 2 (157, 3) - Option 1 gets unique values as we need

In [36]:
# pd.DataFrame(norm_job_dic.items(), columns = ['normalized_job_code', 'title']).to_parquet('../data/raw/norm_job_codes-names.parquet')

#### 1.3 Webscrapping

Finally, we will need to retrieve information about country codes from Eurostat website.

1.3.1 Api Country Codes - More info available here: https://github.com/apilayer/restcountries and Api available here: https://restcountries.eu/rest/v2/alpha/{code}


In [37]:
country_api = []
country_code_list = df_country_info['country_code'].unique().tolist()

In [38]:
for i in country_code_list:
    response = requests.get(f'https://restcountries.eu/rest/v2/alpha/{i}')
    country_json = response.json()
    country_api.append(country_json)
# country_api
# More info available https://github.com/apilayer/restcountries
# country_api

In [80]:
country_api_code = pd.DataFrame(country_api).rename(columns = {'name':'country_name', 'alpha2Code': 'country_code','alpha3Code': 'country_code2'})[['country_name', 'country_code', 'country_code2']]
# country_api_code

In [90]:
country_api_code.to_csv('../data/raw/data_countries_api.csv')

#### Save to CSV

In [93]:
path = '../data/raw/'
data_raw.to_csv(f'{path}data_raw.csv')
country_api_code.to_csv(f'{path}data_countries_api.csv')
jobs2.to_csv(f'{path}data_jobs_api.csv')